In [1]:
from rl.dynamic_programming import V, S, A, extended_vf
from rl import dynamic_programming

In [2]:
from rl import markov_process, markov_decision_process

In [3]:
import operator
from typing import Mapping, Iterator, TypeVar, Tuple, Dict, Iterable

import numpy as np

from rl.distribution import Categorical, Choose, FiniteDistribution
from rl.iterate import converged, iterate
from rl.markov_process import NonTerminal, State
from rl.markov_decision_process import (
    FiniteMarkovDecisionProcess,
    FiniteMarkovRewardProcess,
)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy


In [4]:
from rl.midterm_2022.priority_q import PriorityQueue
from rl.midterm_2022 import grid_maze


In [5]:
import importlib

In [6]:
from time import time

# Problem 1

In this problem we will implement the gaps-based value iteration algorithm mentioned in class.

The gaps-based iteration algorithm proceeds as follows

1. Initialize the value function to zero for all states: $v[s] = 0\ \forall s \in \mathcal{N}$
2. Calculate the gaps for each state: $g[s] = |v[s] - \max_a \mathcal{R}(s,a) + \sum_{s'} \mathcal{P}(s,a,s') \cdot v(s')|$
3. While there is some gap that exceeds a threshold
 - Select the state with the  largest gap: $s_{max} = \arg\max_{s \in \mathcal{N}} g[s]$
 - Update the value function for $s_{max}$: $v[s_{max}] = \max_a \mathcal{R}(s_{max},a) + \sum_{s'}\mathcal{P}(s_{max},a,s') \cdot v(s')$
 -  Update the gap for $s_{max}$: $g[s_{max}] = 0$
 - Oh yeah!
4. Return v

We will test your implementation on a grid maze MDP. We have defined this class in "grid_maze.py", you should  briefly familiarize yourself with that code. In particular pay attention to the difference in reward functions for the two classes "GridMazeMDP_Dense" and "GridMazeMDP_Sparse"

Here is how you can use the classes:

In [7]:
underlying_maze = grid_maze.Maze(10, 10)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0) # goal is at (0, 0)

you can visualize the maze if you wish

In [8]:
print(maze_mdp)

--------------------
|*      | |         |
| +-+-+ + +-+ +-+-+ +
| | | |           | |
| + + + +-+-+ +-+-+-+
|     | |   |   |   |
| +-+-+ + +-+ + +-+ +
| |   |   | | | |   |
| + + +-+-+ +-+-+ +-+
|   | | | | |       |
| +-+-+ + + +-+-+ + +
|                 | |
|-+ +-+-+-+ + +-+ +-+
|         | | |     |
| + +-+ + + +-+-+ +-+
| |   | | |   |   | |
| +-+-+ +-+-+ + +-+ +
|   |   |     |     |
|-+ +-+-+-+ + +-+-+ +
|       |   |   |   |
|-+-+-+-+-+-+-+-+-+-+


you can also visualize a policy on the mdp

In [9]:
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(maze_mdp.print_policy(v2_res[1]))

--------------------
|* < < <|v|> v < < <|
| +-+-+ + +-+ +-+-+ +
|^|v|v|^ < < < < <|^|
| + + + +-+-+ +-+-+-+
|^ < <|^|v <|^ <|> v|
| +-+-+ + +-+ + +-+ +
|^|v <|^ <|v|^|^|v <|
| + + +-+-+ +-+-+ +-+
|^ <|^|v|v|v|> > v <|
| +-+-+ + + +-+-+ + +
|^ < < < < < < < <|^|
|-+ +-+-+-+ + +-+ +-+
|> ^ < < <|^|^|> ^ <|
| + +-+ + + +-+-+ +-+
|^|^ <|^|^|^ <|> ^|v|
| +-+-+ +-+-+ + +-+ +
|^ <|> ^|> > ^|^ < <|
|-+ +-+-+-+ + +-+-+ +
|> ^ < <|> ^|^ <|> ^|
|-+-+-+-+-+-+-+-+-+-+


You will need to make use of the PriorityQueue class in your implementation. A PriorityQueue is an ordered queue which supports the following operations
1. isEmpty(self): check if the queue is empty   
2. contains(self, element): check if the queue contains an element
3. peek(self): peek at the highest priority element in the queue    
4. pop(self): remove and return the highest priority element in the queue    
5. insert(self, element, priority): insert an element into the queue with given priority
6. update(self, element, new_priority): update the priority of an element in the queue
7. delete(self, element): delete an element from the queue

Below are some examples of using the queue

In [10]:
q: PriorityQueue = PriorityQueue()
print(q.isEmpty(), ':', "the queue is empty")
q.insert("a", 1)
print(q.isEmpty(), ':',  "the queue is not empty")
print(q.contains("a"), ':',  "the queue contains a")
print(q.contains("b"), ':',  "the queue does not contain a")
print(q.peek(), ':',  "a is the first element in the queue")
q.insert("b", 0)
print(q.contains("b"), ':',  "the queue now contains b")
print(q.peek(), ':',  "b is now at the front of the queue")
x = q.pop()
print(x, ':',  "we removed b from the queue")
print(q.isEmpty(), ':',  "the queue still nonempty")
print(q.contains("a"), ':',  "the queue still contains a")
print(q.contains("b"), ':',  "the queue does not contain b anymore")
print(q.peek(), ':',  "a is at the front of the queue")
q.insert("c", 5)
print(q.peek(), ':',  "a is still at the front of the queue")
q.update("a", 6)
print(q.peek(), ':',  "after updating a is no longer at the front of the queue")

True : the queue is empty
False : the queue is not empty
True : the queue contains a
False : the queue does not contain a
(1, 'a') : a is the first element in the queue
True : the queue now contains b
(0, 'b') : b is now at the front of the queue
b : we removed b from the queue
False : the queue still nonempty
True : the queue still contains a
False : the queue does not contain b anymore
(1, 'a') : a is at the front of the queue
(1, 'a') : a is still at the front of the queue
(5, 'c') : after updating a is no longer at the front of the queue


### Part 1

In [11]:
from collections import defaultdict


def invert_transition_map(
    mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A]
) -> Mapping[S, Iterable[S]]:
    """Map each state to its set of dependent states."""
    state_action_mapping = mdp.mapping
    inverted_mapping = {state: set() for state in state_action_mapping.keys()}

    for state in state_action_mapping.keys():
        for action in state_action_mapping[state].keys():
            dist = state_action_mapping[state][action]
            
            for reachable_state_reward_pair in dist.table().keys():
                try:
                    inverted_mapping[reachable_state_reward_pair[0]].add(state)
                except KeyError:
                    inverted_mapping[reachable_state_reward_pair[0]] = {state}

    return inverted_mapping


### Part 2

In [12]:
def optimal_reward(
    target_state: S,
    mdp: FiniteMarkovDecisionProcess[S, A],
    v: V[S],
    gamma: float,
) -> float:
    """Calculate the maximum value of Q(s, a) for a fixed s."""
    opt_reward = max(
        mdp.mapping[target_state][a].expectation(
            lambda s_r: s_r[1] + gamma * extended_vf(v, s_r[0])
        )
        for a in mdp.actions(target_state)
    )
    return opt_reward


def gaps_value_iteration(
    mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A],
    gamma: float,
    gaps: PriorityQueue,
) -> Iterator[V[S]]:
    """
    Calculate the value function (V*) of the given MDP by applying the
    update function repeatedly until the values converge.

    """
    dependency_map = invert_transition_map(mdp)
    v_0: V[S] = {s: 0.0 for s in mdp.non_terminal_states}

    def update(v: V[S]) -> V[S]:

        target_state = NonTerminal(gaps.pop())
        v[target_state] = optimal_reward(
            target_state=target_state, mdp=mdp, v=v, gamma=gamma
        )

        for dependent_state in dependency_map[target_state]:
            reward = np.abs(
                v[dependent_state]
                - optimal_reward(
                    target_state=dependent_state,
                    mdp=mdp,
                    v=v,
                    gamma=gamma,
                )
            )

            if reward == 0.0 or reward == -0.0:
                continue
            else:

                if gaps.contains(dependent_state.state):
                    gaps.update(dependent_state.state, -reward)
                else:
                    gaps.insert(dependent_state.state, -reward)

        return v

    return iterate(update, v_0)


def gaps_value_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A], gamma: float
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:

    gaps: PriorityQueue = PriorityQueue()

    v: V[S] = {s: 0.0 for s in mdp.non_terminal_states}

    # populate the queue
    for state in v.keys():
        state_gap = np.abs(optimal_reward(state, mdp, v, gamma) - v[state])
        if state_gap != 0:  # no point adding states with zero gap!
            gaps.insert(state.state, -state_gap)

    # def criterion(x, y):
    #     pass

    # NOTE: Implementing a stopping condition using converged(), which is
    # based on a boolean criterion function does not seem applicable here, because 
    # convergence in gap-value iteration should be defined by the priority queue, 
    # not the state of the value function.  Therefore, I implemented my own stopping 
    # condition:

    for gv_iter in gaps_value_iteration(mdp, gamma, gaps):
        if gaps.isEmpty():
            break

    opt_vf = gv_iter

    opt_policy: markov_decision_process.FiniteDeterministicPolicy[
        S, A
    ] = dynamic_programming.greedy_policy_from_vf(mdp, opt_vf, gamma)

    return opt_vf, opt_policy


In [13]:
v1_res = gaps_value_iteration_result(maze_mdp, 0.9)

# Do not change the code below here, just run it

## Calculating the VF for a maze with sparse rewards

In [14]:
underlying_maze = grid_maze.Maze(50, 50)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [15]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 0.9)
print(time() - start)

0.4800698757171631


In [16]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(time() - start)

6.735416889190674


#### confirming that the value functions are identical

In [17]:
assert v1_res[1] == v2_res[1]

## Calculating the VF for a maze with dense rewards

In [18]:
maze_mdp = grid_maze.GridMazeMDP_Dense(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [19]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 1)
print(time() - start)

15.614510297775269


In [20]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 1)
print(time() - start)

7.1651880741119385


#### confirming that the value functions are identical

In [21]:
assert v1_res[1] == v2_res[1]